In [20]:
import requests
import json
from datetime import datetime, timedelta

# Create timestamps
now = datetime.now()
first_event_time = now - timedelta(minutes=5)  # 5 minutes ago
last_event_time = now

# Build test incident payload
test_incident = {
    "id": f"TEST_{int(now.timestamp())}",
    "timestamp": now.isoformat(),
    "description": json.dumps({
        "rule_id": "834613",
        "rule_description": "1&1 security alert",
        "rule_level": 10,
        "total_events": 2,
        "agent_names": ["Blue Iris"],
        "first_event_timestamp": first_event_time.isoformat(),
        "last_event_timestamp": last_event_time.isoformat(),
        "sample_event": {
            "rule": {
                "id": "834613",
                "description": "Critical Camera Alert",
                "level": 10
            },
            "agent": {
                "name": "Blue Iris",
                "id": "001"
            },
            "timestamp": first_event_time.isoformat(),
            "data": {
                "service": "nginx",
                "status": "stopped",
                "exit_code": 1
            }
        },
        "csv_path": "/incidents/test/events.csv"
    }),
    "acknowledged": False,
    "escalated": False
}

# Send POST request using requests module
response = requests.post(
    url='http://localhost:8000/incidents/',
    headers={'Content-Type': 'application/json'},
    json=test_incident
)

if response.status_code != 200:
    print('Error:', response.status_code, response.text)
else:
    print('Response:', response.json())

Response: {'message': 'Incident created', 'id': 'TEST_1731604758'}


In [1]:
#!/usr/bin/env python3
from auth import register_user, SessionLocal, Base, engine
from fastapi import HTTPException
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def init_db():
    # Create tables
    Base.metadata.create_all(bind=engine)
    
    try:
        # Create admin user
        admin_user = register_user("admin", "admin")
        logger.info("Admin user created successfully")
        return True
    except HTTPException as e:
        if e.detail == "Username already registered":
            logger.info("Admin user already exists")
        else:
            logger.error(f"Error creating admin user: {e.detail}")
        return False
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        return False

if __name__ == "__main__":
    init_db()

INFO:__main__:Admin user created successfully
